Import libraries for data cleaning 

In [1]:
import csv
from tokenize import String

from notebooks.db_connection import get_db_engine
from notebooks.db_connection import execute_query
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

Connection Successful!


Reading CSV file

In [2]:
df_languages = pd.read_csv(r"C:\Users\ricca\Desktop\CSV\languages.csv\languages.csv")

Top records of CSV file: languages.csv

In [3]:
df_languages

,id,type,language
0,1000001,Language,English
1,1000002,Primary language,Korean
2,1000002,Spoken language,English
3,1000002,Spoken language,German
4,1000002,Spoken language,Korean
...,...,...,...
1038757,1941593,Language,Chinese
1038758,1941594,Language,English
1038759,1941595,Language,English
1038760,1941596,Language,Chinese


Checking for total dimension of the csv

In [4]:
df_languages.shape

(1038762, 3)

Rename 'id' column in 'id_language'. "inplace = True" modifies directly the df without creating a copy

In [5]:
df_languages.rename(columns = {"id":"id_movie"} , inplace = True)


In [6]:
df_languages.columns

Index(['id_movie', 'type', 'language'], dtype='object')

Check info about column type

In [7]:
df_languages.dtypes

id_movie     int64
type        object
language    object
dtype: object

Correction of data type of the records

In [8]:
df_languages['id_movie'] = df_languages['id_movie'].astype('Int64')
df_languages['type'] = df_languages['type'].fillna(pd.NA).astype('string')
df_languages['language'] = df_languages['language'].fillna(pd.NA).astype('string')
df_languages.dtypes

id_movie             Int64
type        string[python]
language    string[python]
dtype: object

Check if there are duplicates

In [9]:
df_languages.duplicated().sum()


np.int64(0)

In [10]:
df_languages.drop_duplicates(inplace = True)
df_languages.duplicated().sum()

np.int64(0)

Check the missing values

In [11]:
df_languages.isnull().sum()

id_movie    0
type        0
language    0
dtype: int64

Dropping records with missing value 

In [12]:
df_languages = df_languages.dropna()
df_languages.isnull().sum()

id_movie    0
type        0
language    0
dtype: int64

Remove languages where id_movie doesn't exist

In [13]:
valid_movies = pd.read_csv("movies_valid_ids.csv")
df_languages = df_languages[df_languages["id_movie"].isin(valid_movies["id_movie"])]
df_languages.shape

(1038753, 3)

Connecting to PostgreSQL database and creating the table

In [14]:
engine = get_db_engine()
sql = \
    """CREATE TABLE IF NOT EXISTS languages (
            id_movie INTEGER CHECK (id_movie>0),
            type TEXT CHECK (type<>''),
            language TEXT CHECK (language<>''),

            PRIMARY KEY (id_movie, type, language)
            );
    """
execute_query(sql)

Query Executed Successfully!


Population of the table using .to_sql function

In [15]:
df_languages.to_sql("languages", engine, if_exists="append", index=False)

753